In [1]:
from bs4 import BeautifulSoup
from io import BytesIO

import csv
import datetime
import numpy as np
import pandas as pd
import requests
import time

#參考: 助教的Github; https://www.finlab.tw/Python-時間序列實作！

#STEP 1: 整理要爬的ETF（手動）
crawl_list = ['VGK' , 'EZU' , 'HEDJ' , 'IEUR' , 'FEZ' , 'EWG' , 'IEV' , 'EWU' , 'HEZU' , 'EWP' , 'EWL' , 'DBEU' , 'DFE' , 'EUFN' , 'FEP' , 'EWQ' , 
              'FDD' , 'GREK' ,'EWI' , 'EWD' , 'HEWG' , 'FEU' , 'IEUS' , 'FSZ' , 'GVAL' , 'FGM' , 'EWN' , 'EUSC' , 'NORW' , 'EWO' , 'EDEN' , 'EIRL' , 'FEUZ' , 
              'DXGE' , 'EWUS' , 'EWK' , 'EURL' , 'ADRD' , 'EUDG' , 'EWGS' , 'WBIA' , 'DBEZ' , 'ENOR' , 'EUMV' , 'EFNL' , 'OEUR' , 'PGAL' , 'GXF' , 
              'DBGR' , 'JPEU' , 'SMEZ' , 'EPV' , 'HEWU' , 'DAX' , 'ADRU' , 'HEWP' , 'EUDV' , 'HFXE' , 'ZGBR' , 'FKU' , 'ZDEU' , 'UPV' , 'HEWL' , 'SCID' , 
              'EDOM' , 'DBUK' , 'GRMY' , 'HEWI' , 'EURZ']

#STEP 2: 做一個爬yahoo finance上指定ETF資料的function
today = int(time.time())
today = format(today)
def crawl_price(ETF): 
    #下載自201512/31 0:00(GMT)到程式執行當下的資料
    dl = requests.post('https://query1.finance.yahoo.com/v7/finance/download/' + ETF + '?period1=1451520000&period2=' + today + '&interval=1d&events=history&crumb=PapEKXpAl4Y')
    df = pd.read_csv(BytesIO(dl.content))
    return df

#印出前五行確定有抓到資料
test = crawl_price('VGK')
test.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2015-12-31,50.290001,50.310001,49.880001,49.880001,45.171795,5702600
1,2016-01-04,49.139999,49.180000,48.450001,49.150002,44.510696,6554800
2,2016-01-05,48.840000,48.900002,48.459999,48.860001,44.248062,4352500
3,2016-01-06,47.889999,48.230000,47.799999,48.049999,43.514526,4513200
4,2016-01-07,47.160000,47.660000,47.090000,47.160000,42.708530,9804900


In [2]:
#STEP 3: 建立只有Adj Close的dataframe
def crawl_close(ETF): 
    #下載自201512/31 0:00(GMT)到程式執行當下的資料
    dl = requests.post('https://query1.finance.yahoo.com/v7/finance/download/' + ETF + '?period1=1451520000&period2=' + today + '&interval=1d&events=history&crumb=PapEKXpAl4Y')
    df = pd.read_csv(BytesIO(dl.content))

    df.set_index('Date' , inplace = True)
    #為了讀取Adj Close的欄位把空格去除掉
    df.columns = [c.replace(' ', '') for c in df.columns]
    df_close = df.AdjClose
    df_close = df_close.to_frame()
    
    #把欄位名稱改成ETF的名稱
    df_close = df_close.rename(columns = {'AdjClose' : ETF})
    #df_close.reset_index(inplace = True)
    
    return df_close

#印出前五行確定有抓到資料
test = crawl_close('VGK')
test.head()

,VGK
Date,
2015-12-31,45.171795
2016-01-04,44.510696
2016-01-05,44.248062
2016-01-06,43.514526
2016-01-07,42.708530


In [3]:
#STEP 4: 全部放在一起
final_df = crawl_close('VGK')
print('正在下載資料: VGK')
for ETFs in crawl_list :
    if ETFs != 'VGK' :
        print('正在下載資料: ' + ETFs)
        next_df = crawl_close(ETFs)
        final_df = pd.merge(final_df , next_df , how = 'left' , on = 'Date')

正在下載資料: VGK
正在下載資料: EZU
正在下載資料: HEDJ
正在下載資料: IEUR
正在下載資料: FEZ
正在下載資料: EWG
正在下載資料: IEV
正在下載資料: EWU
正在下載資料: HEZU
正在下載資料: EWP
正在下載資料: EWL
正在下載資料: DBEU
正在下載資料: DFE
正在下載資料: EUFN
正在下載資料: FEP
正在下載資料: EWQ
正在下載資料: FDD
正在下載資料: GREK
正在下載資料: EWI
正在下載資料: EWD
正在下載資料: HEWG
正在下載資料: FEU
正在下載資料: IEUS
正在下載資料: FSZ
正在下載資料: GVAL
正在下載資料: FGM
正在下載資料: EWN
正在下載資料: EUSC
正在下載資料: NORW
正在下載資料: EWO
正在下載資料: EDEN
正在下載資料: EIRL
正在下載資料: FEUZ
正在下載資料: DXGE
正在下載資料: EWUS
正在下載資料: EWK
正在下載資料: EURL
正在下載資料: ADRD
正在下載資料: EUDG
正在下載資料: EWGS
正在下載資料: WBIA
正在下載資料: DBEZ
正在下載資料: ENOR
正在下載資料: EUMV
正在下載資料: EFNL
正在下載資料: OEUR
正在下載資料: PGAL
正在下載資料: GXF
正在下載資料: DBGR
正在下載資料: JPEU
正在下載資料: SMEZ
正在下載資料: EPV
正在下載資料: HEWU
正在下載資料: DAX
正在下載資料: ADRU
正在下載資料: HEWP
正在下載資料: EUDV
正在下載資料: HFXE
正在下載資料: ZGBR
正在下載資料: FKU
正在下載資料: ZDEU
正在下載資料: UPV
正在下載資料: HEWL
正在下載資料: SCID
正在下載資料: EDOM
正在下載資料: DBUK
正在下載資料: GRMY
正在下載資料: HEWI
正在下載資料: EURZ


In [4]:
#STEP 5: 加回index並印出結果
final_df.reset_index(inplace = True)
final_df

,Date,VGK,EZU,HEDJ,IEUR,FEZ,EWG,IEV,EWU,HEZU,...,FKU,ZDEU,UPV,HEWL,SCID,EDOM,DBUK,GRMY,HEWI,EURZ
0,2015-12-31,45.171795,32.330105,49.842030,38.539001,31.393398,24.428692,36.811104,28.877220,23.706516,...,36.628689,NaN,38.414822,20.360424,20.610361,21.825270,16.026497,21.597849,14.375425,21.648333
1,2016-01-04,44.510696,31.804184,49.119549,37.909454,30.727783,23.841057,36.260452,28.412039,23.421892,...,36.008175,NaN,36.924759,20.567041,20.610361,21.493631,16.026497,21.597849,14.243299,21.648333
2,2016-01-05,44.248062,31.527388,48.999142,37.663109,30.454241,23.617199,36.031013,28.340473,23.375982,...,36.099434,NaN,36.521000,20.567041,20.610361,21.359188,15.634013,21.597849,14.329180,21.648333
3,2016-01-06,43.514526,31.010700,48.026566,37.006199,29.934515,23.346701,35.425297,27.821609,22.944456,...,35.287281,NaN,35.280880,20.567041,20.610361,20.893103,15.634013,21.597849,13.846916,21.648333
4,2016-01-07,42.708530,30.577044,46.646439,36.386684,29.515083,22.880329,34.782867,27.052267,22.283392,...,34.484268,NaN,33.886951,20.567041,20.610361,20.722805,15.110700,21.597849,13.595876,21.648333
5,2016-01-08,42.337231,30.300249,46.294460,36.002571,29.214186,22.656469,34.434124,26.873350,22.118124,...,34.183128,NaN,33.492805,19.688917,20.610361,20.722805,15.067091,21.597849,13.549630,21.648333
6,2016-01-11,42.464016,30.466324,46.850220,36.057316,29.378311,22.833691,34.553432,26.944918,22.393568,...,34.320007,NaN,32.992912,19.688917,20.610361,20.722805,15.067091,21.597849,13.543026,21.648333
7,2016-01-12,42.817200,30.881529,47.378185,36.404026,29.743032,23.197464,34.828758,26.962809,22.696554,...,34.164875,NaN,34.482975,19.688917,20.610361,20.660059,15.067091,21.597849,13.595876,21.648333
8,2016-01-13,42.002155,30.272570,46.238884,35.701488,29.123003,22.619160,34.232220,26.515514,22.191576,...,33.562614,NaN,33.165951,20.162420,20.610361,20.660059,15.067091,21.597849,13.443929,21.648333
9,2016-01-14,42.545521,30.558594,46.831688,36.075565,29.478609,22.880329,34.672741,27.016485,22.421114,...,33.562614,NaN,33.502415,19.852491,20.610361,20.292572,14.936258,21.597849,13.609089,21.648333
